In [1]:
import requests
import json
from datetime import datetime
from trust_algorithm import update_blacklist

Trust algorithm ran successfully


In [2]:
url = 'http://127.0.0.1:8000/locationConsensus/'
file = open('test_data.csv')

for line in file:
    # read in the csv (extra steps due to list as a column)
    splits = line.split(',')
    from_user = splits[0]
    time = datetime.strptime(splits[-1].strip(), '%Y-%m-%d %H:%M:%S')
    users = []
    for user in splits[1:-1]:
        users.append(user.replace("'", '').replace("[", '').replace("]", '').replace(" ", ''))

    toSend = {
        'from_user':from_user, 
        'spotted_users': json.dumps(users),
        'time':time
    } 

    out1 = requests.post(url + 'users/', data={'userID':from_user}) # add the user to the database
    out2 = requests.post(url + 'interactions/', data=toSend) # add the interaction to the database

print(out1.text)
print(out2.text)

{"userID":"alex"}
{"from_user":"alex","spotted_users":"[\"alan\"]","time":"2023-02-09T01:09:27Z"}


In [3]:
from datetime import timezone, timedelta
import pandas as pd

# urls of site
base_url = "https://dsc180-decentralized-location.herokuapp.com/locationConsensus/"
interaction_url = base_url + "interactions/"
blacklist_url = base_url + "blacklist/"

# fields
cuttoff = 0.5 # cutoff percentage to be marked for blacklist

# gets interaction data
data = requests.get(interaction_url).json()
df = pd.DataFrame(data)

spotted_percentage = df.groupby("spotted_users")["from_user"].count() / df["from_user"].nunique()
flagged_users = spotted_percentage[spotted_percentage > cuttoff]

update_blacklist(flagged_users.index)

In [4]:
requests.get(url + 'blacklist/').text

'[]'